## I don't understand why I get a Submission Scoring Error. If anyone knows, please let me know.

In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
import gresearch_crypto
import re

In [ ]:
import pickle
f = open('../input/models1/models1', 'rb')
models = pickle.load(f)

f = open('../input/target-corr/target_corr', 'rb')
corr = pickle.load(f)

In [ ]:
import datetime
def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']


def add_time(df):
  df["date"] = df.index.map(datetime.datetime.fromtimestamp)

  df["year"] = df["date"].map(lambda x:x.year)
  df['month'] = df['date'].map(lambda x:x.month)
  df['day'] = df['date'].map(lambda x:x.day)
  df['hour'] = df['date'].map(lambda x:x.hour)
  df['minute'] = df['date'].map(lambda x:x.minute)

  df["sin_month"] = (np.sin(2 * np.pi * df.month/12))
  df["cos_month"] = (np.cos(2 * np.pi * df.month/12))
  df["sin_day"] = (np.sin(2 * np.pi * df.day/31))
  df["cos_day"] = (np.cos(2 * np.pi * df.day/31))
  df["sin_hour"] = (np.sin(2 * np.pi * df.hour/24))
  df["cos_hour"] = (np.cos(2 * np.pi * df.hour/24))

  df['date_top'] = df['date'].dt.strftime('%Y')+'/1/1 0:0:0'
  df['date_top'] = pd.to_datetime(df['date_top'])
  df['sec_in_year'] = (df['date'].view('int64') - df['date_top'].view('int64')) // 10**9
  df['sec_in_year'].head()

  df["sin_year"] = (np.sin(2 * np.pi * df.sec_in_year/(365*24*60*60)))
  df["cos_year"] = (np.cos(2 * np.pi * df.sec_in_year/(365*24*60*60)))
  #df = df.add_suffix('_s')
  #df.drop("datetime_0_s",axis=1,inplace=True)
  df.drop('date_top',axis=1,inplace=True)
  df = df.add_suffix("_s")

  return df

def get_features(df):
  df_all = pd.DataFrame()
  for id in range(14):

    df_ = df.groupby('Asset_ID').get_group(id)[['timestamp', 'Count', 'Open', 'High', 'Low', 'Close','Volume', 'VWAP']]
    df_['Upper_Shadow'] = upper_shadow(df_)
    df_['Lower_Shadow'] = lower_shadow(df_)
    df_ = df_.set_index("timestamp")
    
    df_ = df_.add_suffix("_"+str(id))

    df_all = pd.concat([df_all,df_],axis=1)

    #y = df_train.groupby('Asset_ID').get_group(id)["Target"] #提出時は消す
    ##y = y.add_suffix("_"+str(id))
    #Target = pd.concat([Target,y],axis=1)

  df_all = add_time(df_all)

  return df_all

#個別で求める。
def get_features2(df,id):
  y = corr.columns[id]
  best = corr[y].drop(y).sort_values(ascending=False,key=abs).index[0]

  best_ind = re.sub(r"\D", "", best)

  col_id = [column for column in df.columns if "_"+str(id)+"_s" in column]
  col_best = [column for column in df.columns if "_"+str(best_ind)+"_s" in column]
  time = ['month_s', 'day_s', 'hour_s', 'minute_s', 'sin_month_s', 'cos_month_s','sin_day_s', 'cos_day_s', 'sin_hour_s', 'cos_hour_s']

  df = df[col_id+col_best+time]

  return df

In [ ]:
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

for i, (df_test, df_pred) in enumerate(iter_test):
    x = get_features(df_test)
    row_ids = df_test.row_id
    for id in df_test.Asset_ID.unique():
        model = models[id]
        x_test = get_features2(x,id)

        row_id = row_ids[id]
        
             
        pred = model.predict(x_test)
        #pred = 0
        
        #x_test['row_id'] =row_id
      
        df_pred.loc[df_pred['row_id'] == row_id, 'Target'] = pred
        
        
    env.predict(df_pred)